In [1]:
from keras.preprocessing import image
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, MaxPooling2D, Flatten, Conv2D

Using TensorFlow backend.


In [ ]:
def generator(dir, gen = image.ImageDataGenerator(rescale = 1./255), shuffle = True,
            batch_size = 1, target_size=(24, 24), class_mode = 'categorical'):
    return gen.flow_from_directory(dir,batch_size=batch_size, shuffle=shuffle, color_mode='grayscale',
                                   class_mode=class_mode, target_size=target_size)

In [ ]:
BS = 32
TS = (24, 24)

In [ ]:
train_batch = generator('/Users/sachinsingh/Downloads/dataset_new/train/', shuffle=True, batch_size = BS, target_size = TS)
valid_batch = generator('/Users/sachinsingh/Downloads/dataset_new/test/', shuffle=True, batch_size = BS, target_size = TS)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(24,24,1)))

In [ ]:
model.add(MaxPooling2D(pool_size=(1,1)))

In [ ]:
model.add(Conv2D(32, (3, 3),activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(1,1)))

In [ ]:
model.add(Conv2D(64, (3, 3),activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(1,1)))

In [ ]:
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(124, activation='relu'))

In [ ]:
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
SPE = len(train_batch.classes)//BS

In [ ]:
SPE

In [ ]:
VS = len(valid_batch.classes)//BS

In [ ]:
model.fit_generator(train_batch, validation_data=valid_batch, epochs=15, steps_per_epoch=SPE , validation_steps=VS)

In [ ]:
model.save('/Users/sachinsingh/Downloads/dataset_new/drive_model.h5', overwrite=True)

In [37]:
import cv2

In [38]:
from pygame import mixer

In [39]:
model1 = load_model('/Users/sachinsingh/Downloads/dataset_new/drive_model.h5')

In [40]:
import numpy as np

In [41]:
face = cv2.CascadeClassifier('/Users/sachinsingh/Downloads/cascade/haarcascade_frontalface_alt.xml')

In [42]:
leye = cv2.CascadeClassifier('/Users/sachinsingh/Downloads/cascade/haarcascade_lefteye_2splits.xml')

In [43]:
reye = cv2.CascadeClassifier('/Users/sachinsingh/Downloads/cascade/haarcascade_righteye_2splits.xml')

In [44]:
mixer.init()
sound = mixer.Sound('/Users/sachinsingh/Downloads/135613__danielnieto7__alert.wav')

In [47]:
score = 0
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
rpred = []
lpred = []

In [50]:
cap = cv2.VideoCapture(1)

while True:
    
    ret, frame = cap.read()
    height, weight = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,0), (0, 0), (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        
        cv2.rectangle(frame, (x, y) , (x+w,y+h) , (100,100,100) , 1)
        
    for (x,y,w,h) in right_eye:
        
        r_eye = frame[y:y+h,x:x+w]
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye = r_eye/255
        r_eye = r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye, axis=0)
        rpred = model1.predict_classes(r_eye)
        break
    
    for (x,y,w,h) in left_eye:
        
        l_eye = frame[y:y+h, x:x+w]
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye = l_eye/255
        l_eye = l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye, axis=0)
        lpred = model1.predict_classes(l_eye)
        break
    
#     print(lpred[0], rpred[0])
    
    if (lpred[0] == 0 & rpred[0] == 0) :
        score=score+1
        cv2.putText(frame, "Closed", (10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)

    else:
        score=score-2
        cv2.putText(frame, "Open", (10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    if(score < 0):
        score=0
        
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1, (255,255,255), 1, cv2.LINE_AA)
    
    if(score > 10):
        #person is feeling sleepy so we beep the alarm
        try:
            sound.play()
        except:  
            pass
        
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

cap.release()
cv2.destroyAllWindows()        

OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


AttributeError: 'NoneType' object has no attribute 'shape'